In [ ]:
import sys
sys.path.append("..")

# deeptrack - Tracking point particles

This notebook demonstrates how to track point particles with a convolutional neural network using DeepTrack.

Specifically, this tutotial explains how to: 
* Define the procedure to generate trainign images
* Extract information from an image to use as labels
* Define and train a model
* Visually evaluate the result

## 1. Setup

Imports needed for this tutorial.

In [ ]:
from deeptrack.scatterers import PointParticle
from deeptrack.optics import OpticalDevice
from deeptrack.generators import Generator
from deeptrack.models import convolutional

import numpy as np
import matplotlib.pyplot as plt

## 2. Define the particle

For this example, we consider a point particle (i.e. a point light source). A point particle is entirely defined by its intensity and its position. A point particle is an instance of the class PointParticle.

A point particle is controlled by the following parameters:

* intensity: The intensity of the point particle

* position: The position of the point particle

* position_unit: "pixel" or "meter"

In [ ]:
particle = PointParticle(                                         
    intensity=100,
    position=(32, 16),
    position_unit="pixel"
)

## 3. Define the optics 

Next, we need to define the properties of the optical system. This is done using an instance of the class Optics, which takes a set of light sources (particles) and convolves them with the pupil function (point spread function) of the otical system.

The optics is controlled by the following parameters:

* NA: The numerical aperature

* pixel_size: The pixel to meter conversion factor (m/px)

* wavelength: The wavelength of the lightsource (m)

* mode: "coherent" or "incoherent" light emitted by the object

* ROI: Region of interest that is imaged (to avoid wrap-around effects when Fourier-tranforming)

* upscale: upscale factor for the pupil function (increases accuracy and computational cost).

In [ ]:
optics = OpticalDevice(
    NA=0.7,                
    pixel_size=0.1e-6,     
    wavelength=680e-9
)

## 4. Generate and plot the image

Finally, we have everything needed to create an image! 

To generate an image, we call the optical device (here, `optics`) with the object we want to image (here, `particle`). This returns a new object (here, `image_of_particle`) corresponding to the image.

To visualize the resulting image will see, we can call the method `.plot()`. This will resolve the image and plot it using matplotlib.

In [ ]:
image_of_particle = optics(particle)

image_of_particle.plot(shape=(64, 64), cmap='gray')

## 5. Randomize the particle position

We can generate particles with random positions by passing a lambda function to the keyword argument `position`, which returns a pair of random numbers representing the particle position. 

In the example below, we also retrieve and plot the position of the particle.

In [ ]:
particle = PointParticle(                                         
    intensity=100,
    position=lambda: 10 + np.random.rand(2) * 44,
    position_unit="pixel"
)

input_image = np.zeros((64, 64))
image_of_particle = optics(particle)
output_image = image_of_particle.resolve(input_image)

position_of_particle = np.array(particle.properties["position"].current_value)

plt.imshow(output_image, cmap='gray')
plt.scatter(position_of_particle[0], position_of_particle[1])
plt.show()

## 6. Define the neural network model

We will use a predefined neural network model to track the particle obtained calling the function `convolutional`. This model is a convolutional neural network with a dense top. It receives an input of shape (64, 64, 1) and outputs two values (x and y position of the particle).

In [ ]:
model = convolutional(
    input_shape=(64, 64, 1), 
    number_of_outputs=2
)

## 7. Define image generator

Generators are objects that feed models with images and their corresponging labels during training. They are created by calling `.generate()` on an instance of the class Generator. This method takes the following inputs:
* feature: A feature that resolves images used to train a model
* label_function: A function that takes an image as input and returns the label for that image
* shape: The shape of the output image
* batch_size: The number of images per batch

In [ ]:
# Function that retireves the position of a particle 
# and divides it by 64 to get values between 0 and 1
def get_position_of_particle(image):
    return np.array(particle.properties["position"].current_value / 64)

generator = Generator().generate(
    image_of_particle, 
    get_position_of_particle, 
    shape=(64, 64), 
    batch_size=4
)

## 8. Train the model

The model is trained by calling `.fit()` with the generator we defined in the previous step.

In [ ]:
model.fit(
    generator,
    epochs=1000,
    steps_per_epoch=64
)

## 9. Visualize the model performance

We can now use the trained model to measure the particle position in images previously unseen by the model.

In [ ]:
images, real_positions = next(generator)

mesured_positions = model.predict(images)

for i in range(images.shape[0]):
    
    image = np.squeeze(images[i])
    plt.imshow(image, cmap='gray')
    
    mesured_position_x = prediction[i, 0] * 64
    mesured_position_y = prediction[i, 1] * 64    
    plt.scatter(mesured_position_x, mesured_position_y)
    
    plt.show()